## Which model is the best?

Follow this link: https://www.kaggle.com/code/jhoward/which-image-models-are-best/

`timm` (Pytorch Image Model) is a libray of Ross Wightman which provides the state-of-the-art pretrained computer vision models.
While HF Transformers is quite specific for NLP, `timm` is for comupter vision

In [1]:
!pip freeze | grep timm

timm==0.9.2


### Data
Ross regularly benchmarks new models as they are added to timm, and puts the results in a CSV in the project's GitHub repo. 

In [2]:
! git clone --depth 1 https://github.com/rwightman/pytorch-image-models.git
%cd pytorch-image-models/results

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 566, done.
remote: Counting objects: 100% (566/566), done.
remote: Compressing objects: 100% (408/408), done.
remote: Total 566 (delta 221), reused 348 (delta 152), pack-reused 0
Receiving objects: 100% (566/566), 2.49 MiB | 11.37 MiB/s, done.
Resolving deltas: 100% (221/221), done.
/Users/hatruong/hagit/fastai-playbook/pytorch-image-models/results


We use pandas to read result `results-imagenet.csv`

In [10]:
import pandas as pd
df_result = pd.read_csv("results-imagenet.csv")
df_result.head()

,model,top1,top1_err,top5,top5_err,param_count,img_size,crop_pct,interpolation
0,eva02_large_patch14_448.mim_m38m_ft_in22k_in1k,90.052,9.948,99.048,0.952,305.08,448,1.0,bicubic
1,eva02_large_patch14_448.mim_in22k_ft_in22k_in1k,89.966,10.034,99.012,0.988,305.08,448,1.0,bicubic
2,eva_giant_patch14_560.m30m_ft_in22k_in1k,89.786,10.214,98.992,1.008,"1,014.45",560,1.0,bicubic
3,eva02_large_patch14_448.mim_in22k_ft_in1k,89.624,10.376,98.950,1.050,305.08,448,1.0,bicubic
4,eva02_large_patch14_448.mim_m38m_ft_in1k,89.570,10.430,98.922,1.078,305.08,448,1.0,bicubic


In [12]:
df_result['model_org'] = df_result['model']
df_result['model'] = df_result['model'].str.split('.').str[0]
df_result.head()

,model,top1,top1_err,top5,top5_err,param_count,img_size,crop_pct,interpolation,model_org
0,eva02_large_patch14_448,90.052,9.948,99.048,0.952,305.08,448,1.0,bicubic,eva02_large_patch14_448.mim_m38m_ft_in22k_in1k
1,eva02_large_patch14_448,89.966,10.034,99.012,0.988,305.08,448,1.0,bicubic,eva02_large_patch14_448.mim_in22k_ft_in22k_in1k
2,eva_giant_patch14_560,89.786,10.214,98.992,1.008,"1,014.45",560,1.0,bicubic,eva_giant_patch14_560.m30m_ft_in22k_in1k
3,eva02_large_patch14_448,89.624,10.376,98.950,1.050,305.08,448,1.0,bicubic,eva02_large_patch14_448.mim_in22k_ft_in1k
4,eva02_large_patch14_448,89.570,10.430,98.922,1.078,305.08,448,1.0,bicubic,eva02_large_patch14_448.mim_m38m_ft_in1k


## How does a neural net really work?

I follow this link: https://www.kaggle.com/code/jhoward/how-does-a-neural-net-really-work

### Fitting a function with gradient descent

TODO: 
- Explain what is a layer
- Explain what is gradient descent

In [ ]:
from ipywidgets import interact
from fastai.basics import *

plt.rc('figure', dpi=90)

def plot_function(f, title=None, min=-2.1, max=2.1, color='r', ylim=None):
    x = torch.linspace(min,max, 100)[:,None]
    if ylim: plt.ylim(ylim)
    plt.plot(x, f(x), color)
    if title is not None: plt.title(title)
